In [79]:
import pandas as pd

from utils.audio_dataset_utils import train_val_dataset
from network_models.soundsream_models_and_utils.ss_model_dim_red import SSDimRedModel
from network_models.soundsream_models_and_utils.ss_model_conv import SSConvModel3Sec
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
from torch.utils.data import DataLoader

batch_size = 1

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings.pkl", device="cuda")

json_path =" /home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/"

modelDimRed = SSDimRedModel().cuda()
modelConv = SSConvModel3Sec(xSize=512, ySize=175).cuda()



In [80]:
modelConv.load_state_dict(torch.load("../content/soundstream/experiments/nr2Run_Nr_0_conv/emo_reco_best_ep216_acc_62"))
modelDimRed.load_state_dict(torch.load("../content/soundstream/experiments/nr2Run_Nr_0_dimred/emo_reco_500.pth"))

<All keys matched successfully>

In [102]:
from numpy import ndarray


def generateParameters(device, model, dataset, labels):
    loader = DataLoader(dataset, shuffle=False, batch_size=1)
    d ={}

    for label in labels:
        d.update({label: []})

    emo_angry = []
    emo_disgust = []
    emo_fear = []
    emo_happy = []
    emo_neutral = []
    emo_sad = []
    emo_surprise = []

    with torch.no_grad():
        for batch, (X, z) in enumerate(loader):
            X, z = X.to(device),  z.to(device)
            dims, pred = model(X, return_with_dims= True)
            label = data_set.encoded_dataset.label_list[z[0].nonzero().cpu().numpy()[0][0]]
            dims = dims.cpu().numpy()[0]
            d[label].append(dims)
            #match (labels):
            #    case "neutral":
            #        emo_neutral.append(dims)
            #    case "happy":
            #        emo_happy.append(dims)
            #    case "sad":
            #        emo_sad.append(dims)
            #    case "angry":
            #        emo_angry.append(dims)
            #    case "fear":
            #        emo_fear.append(dims)
            #    case "disgust":
            #        emo_disgust.append(dims)
            #    case "surprise":
            #        emo_surprise.append(dims)
            if batch % 500 == 0:
                print(f"{batch*batch_size} of {len(dataset)}")

    return d
    return emo_angry, emo_disgust, emo_fear, emo_happy, emo_neutral, emo_sad, emo_surprise


def genAvgFull(fulldata: list[dict], label_list, modelNameList):
    fullDict = {}
    for label in label_list:
        fullDict.update({label: genAvgAndVar([fd[label] for fd in fulldata], modelNameList)})
    return fullDict

def genAvgAndVar(data: list[list[ndarray[float]]], modelNameList):
    d = {}
    for i in range(len(modelNameList)):
        mean = np.mean(np.asarray(data[i]), axis=0)
        var = np.var(np.asarray(data[i]), axis=0)
        d.update({modelNameList[i] : pd.Series([mean, var], index=["mean", "var"])})
    return pd.DataFrame(d)


def generateJson(data):
    str = "{"
    for i in range(len(data)):
        str += f"x{i}: {data[i]},"
    str += "}"
    return(str)

def writeJson(emotion, string):
    file = open(json_path+f"{emotion}.json", "w")
    file.write(string)
    file.close()

In [100]:
emo = generateParameters("cuda", modelDimRed, data_set, data_set.encoded_dataset.label_list)
conv_emo = generateParameters("cuda", modelConv, data_set, data_set.encoded_dataset.label_list)


/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0 of 6092
500 of 6092
1000 of 6092
1500 of 6092
2000 of 6092
2500 of 6092
3000 of 6092
3500 of 6092
4000 of 6092
4500 of 6092
5000 of 6092
5500 of 6092
6000 of 6092
0 of 6092
500 of 6092
1000 of 6092
1500 of 6092
2000 of 6092
2500 of 6092
3000 of 6092
3500 of 6092
4000 of 6092
4500 of 6092
5000 of 6092
5500 of 6092
6000 of 6092


In [103]:
import numpy as np

genAvgFull([emo, conv_emo], data_set.encoded_dataset.label_list, ["red", "conv"])

# emo_angry_avg = np.mean(np.asarray(emo_angry), axis=0)
# emo_angry_var = np.var(np.asarray(emo_angry), axis=0)
# emo_disgust_avg = np.mean(np.asarray(emo_disgust), axis=0)
# emo_fear_avg = np.mean(np.asarray(emo_fear), axis=0)
# emo_happy_avg = np.mean(np.asarray(emo_happy), axis=0)
# emo_neutral_avg = np.mean(np.asarray(emo_neutral), axis=0)
# emo_sad_avg = np.mean(np.asarray(emo_sad), axis=0)
# emo_surprise_avg = np.mean(np.asarray(emo_surprise), axis=0)
#
#
# conv_emo_angry_avg = np.mean(np.asarray(conv_emo_angry), axis=0)
# conv_emo_angry_var = np.var(np.asarray(conv_emo_angry), axis=0)
# conv_emo_disgust_avg = np.mean(np.asarray(conv_emo_disgust), axis=0)
# conv_emo_fear_avg = np.mean(np.asarray(conv_emo_fear), axis=0)
# conv_emo_happy_avg = np.mean(np.asarray(conv_emo_happy), axis=0)
# conv_emo_neutral_avg = np.mean(np.asarray(conv_emo_neutral), axis=0)
# conv_emo_sad_avg = np.mean(np.asarray(conv_emo_sad), axis=0)
# conv_emo_surprise_avg = np.mean(np.asarray(conv_emo_surprise), axis=0)

{'angry':                                                    red  \
 mean  [-0.66945684, -0.5150308, 0.64851785, 0.7231073]   
 var    [0.14429769, 0.17769252, 0.08217345, 0.1094079]   
 
                                                    conv  
 mean  [0.92897815, -0.92863935, -0.9120094, 0.38886377]  
 var    [0.0807195, 0.08701974, 0.06680432, 0.036286253]  ,
 'disgust':                                                     red  \
 mean  [0.60760134, -0.75726825, 0.58009577, -0.5529675]   
 var   [0.09445228, 0.08733735, 0.117719695, 0.13388322]   
 
                                                    conv  
 mean    [-0.8281549, -0.9411225, 0.89458096, 0.9407132]  
 var   [0.08310469, 0.058798447, 0.07807269, 0.03695233]  ,
 'fear':                                                    red  \
 mean   [0.68669516, 0.59777653, 0.42156133, 0.5944756]   
 var   [0.073097914, 0.1587208, 0.13954547, 0.10557774]   
 
                                                    conv  
 mean     [0.9076

In [90]:
print(genAvgAndVar([emo_angry, conv_emo_angry], ["red", "conv"]))

                                                   red  \
mean    [-0.6666686, -0.5079258, 0.6491481, 0.7223425]   
var   [0.14889714, 0.17890035, 0.08293114, 0.11087131]   

                                                   conv  
mean     [0.9240394, -0.934461, -0.91103846, 0.3788689]  
var   [0.08614871, 0.07793402, 0.06794015, 0.039782394]  


In [87]:
print(f"neutral :  redDim: {emo_neutral_avg}, conv: {conv_emo_neutral_avg}")
print(f"happy   :  redDim: {emo_happy_avg}, conv: {conv_emo_happy_avg}")
print(f"sad     :  redDim: {emo_sad_avg}, conv: {conv_emo_sad_avg}")
print(f"angry   :  redDim: {emo_angry_avg}, conv: {conv_emo_angry_avg}")
print(f"fear    :  redDim: {emo_fear_avg}, conv: {conv_emo_fear_avg}")
print(f"disgust :  redDim: {emo_disgust_avg}, conv: {conv_emo_disgust_avg}")
print(f"surprise:  redDim: {emo_surprise_avg}, conv: {conv_emo_surprise_avg}")



neutral :  redDim: [ 0.6628585   0.73390675 -0.73620534 -0.6215517 ], conv: [-0.43281537 -0.9381207   0.25976825 -0.89507014]
happy   :  redDim: [-0.5877671  -0.6784905  -0.14874858 -0.49661595], conv: [-0.9198704   0.8013732  -0.94068444  0.5576449 ]
sad     :  redDim: [ 0.7614937   0.5211136   0.55389595 -0.6890108 ], conv: [-0.33636513  0.8152136   0.9197464  -0.55772597]
angry   :  redDim: [-0.6711606  -0.5146098   0.64993006  0.72506434], conv: [ 0.9131994  -0.933766   -0.9017556   0.38519347]
fear    :  redDim: [0.6828919  0.6016554  0.42267424 0.5859955 ], conv: [ 0.90904963  0.89146507 -0.6525785   0.9084787 ]
disgust :  redDim: [ 0.6011055 -0.7597309  0.5779803 -0.5371726], conv: [-0.8343913  -0.93599176  0.89802724  0.94474196]
surprise:  redDim: [ 0.6730314  -0.6501411   0.48391184  0.6193821 ], conv: [ 0.8756019   0.8056791  -0.9436177  -0.93016684]


In [21]:
print("neutral:" + generateJson(conv_emo_neutral_avg))
print("happy:" + generateJson(conv_emo_happy_avg))
print("sad:" + generateJson(conv_emo_sad_avg))
print("angry:" + generateJson(conv_emo_angry_avg))
print("fear:" + generateJson(conv_emo_fear_avg))
print("disgust:" + generateJson(conv_emo_disgust_avg))
print("surprise:" + generateJson(conv_emo_surprise_avg))

neutral:{x0: -0.4242432713508606,x1: -0.9272955656051636,x2: 0.2532370984554291,x3: -0.9015499949455261,}
happy:{x0: -0.9147626757621765,x1: 0.8096162676811218,x2: -0.9410518407821655,x3: 0.5426420569419861,}
sad:{x0: -0.34070485830307007,x1: 0.8153109550476074,x2: 0.9143989682197571,x3: -0.5619972348213196,}
angry:{x0: 0.9265413284301758,x1: -0.9317540526390076,x2: -0.9140453934669495,x3: 0.38750553131103516,}
fear:{x0: 0.9012864232063293,x1: 0.8959128856658936,x2: -0.6553657650947571,x3: 0.9066473245620728,}
disgust:{x0: -0.8301857113838196,x1: -0.9352936148643494,x2: 0.8977404236793518,x3: 0.9390570521354675,}
surprise:{x0: 0.8728251457214355,x1: 0.8140773177146912,x2: -0.9374096989631653,x3: -0.9270768165588379,}
